In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

# Cargar los datos
tesla_data = pd.read_csv('dataset/acciones_tesla_mensuales.csv')
toyota_data = pd.read_csv('dataset/acciones_toyota_mensuales.csv')

# Filtrar los datos a partir de 2010
tesla_data = tesla_data[tesla_data['fecha'] >= 'Jan-2010']
toyota_data = toyota_data[toyota_data['fecha'] >= 'Jan-2010']

# Convertir la columna 'date' a formato datetime
tesla_data['fecha'] = pd.to_datetime(tesla_data['fecha'])
toyota_data['fecha'] = pd.to_datetime(toyota_data['fecha'])

# Configurar los índices de fecha
tesla_data = tesla_data.set_index('fecha')
toyota_data = toyota_data.set_index('fecha')

# Dividir los datos en conjuntos de entrenamiento y prueba
train_end = pd.Timestamp('2021-12-31')
test_start = pd.Timestamp('2022-01-01')
test_end = pd.Timestamp('2023-12-31')
predict_start = pd.Timestamp('2024-01-01')

tesla_train = tesla_data[:train_end]
tesla_test = tesla_data[test_start:test_end]
tesla_predict = tesla_data[predict_start:]

toyota_train = toyota_data[:train_end]
toyota_test = toyota_data[test_start:test_end]
toyota_predict = toyota_data[predict_start:]



In [4]:
# ARIMA Seasonal
def arima_seasonal(train, test, predict):
    model = ARIMA(train['precio'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=len(test) + len(predict))[len(test):]
    return forecast

tesla_arima_seasonal = arima_seasonal(tesla_train, tesla_test, tesla_predict)
toyota_arima_seasonal = arima_seasonal(toyota_train, toyota_test, toyota_predict)

# ARIMA No Seasonal
def arima_non_seasonal(train, test, predict):
    model = ARIMA(train['precio'], order=(1, 1, 1))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=len(test) + len(predict))[len(test):]
    return forecast

tesla_arima_non_seasonal = arima_non_seasonal(tesla_train, tesla_test, tesla_predict)
toyota_arima_non_seasonal = arima_non_seasonal(toyota_train, toyota_test, toyota_predict)

c:\Users\esposito\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\esposito\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\esposito\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\esposito\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few ob

LinAlgError: Schur decomposition solver error.

In [ ]:
# STL
def stl_forecast(train, test, predict):
    stl = seasonal_decompose(train['close'], period=12)
    trend = stl.trend
    seasonal = stl.seasonal
    residual = stl.resid
    forecast = trend.iloc[-1] + seasonal.iloc[-12:].sum() + residual.mean()
    return pd.Series([forecast] * len(predict), index=predict.index)

tesla_stl = stl_forecast(tesla_train, tesla_test, tesla_predict)
toyota_stl = stl_forecast(toyota_train, toyota_test, toyota_predict)


In [ ]:
# Forecasting Lineal
def linear_forecast(train, predict):
    model = SimpleExpSmoothing(train['close'])
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=len(predict))
    return forecast

tesla_linear = linear_forecast(tesla_train, tesla_predict)
toyota_linear = linear_forecast(toyota_train, toyota_predict)

In [ ]:
# Visualizar los resultados
fig, ax = plt.subplots(2, 2, figsize=(16, 10))

ax[0, 0].plot(tesla_data, label='Datos reales')
ax[0, 0].plot(tesla_train.index, tesla_train['close'], label='Entrenamiento')
ax[0, 0].plot(tesla_test.index, tesla_test['close'], label='Prueba')
ax[0, 0].plot(tesla_predict.index, tesla_arima_seasonal, label='ARIMA Seasonal')
ax[0, 0].plot(tesla_predict.index, tesla_arima_non_seasonal, label='ARIMA No Seasonal')
ax[0, 0].plot(tesla_predict.index, tesla_stl, label='STL')
ax[0, 0].plot(tesla_predict.index, tesla_linear, label='Forecasting Lineal')
ax[0, 0].set_title('Predicción de precios de acciones de Tesla')
ax[0, 0].legend()

ax[0, 1].plot(toyota_data, label='Datos reales')
ax[0, 1].plot(toyota_train.index, toyota_train['close'], label='Entrenamiento')
ax[0, 1].plot(toyota_test.index, toyota_test['close'], label='Prueba')
ax[0, 1].plot(toyota_predict.index, toyota_arima_seasonal, label='ARIMA Seasonal')
ax[0, 1].plot(toyota_predict.index, toyota_arima_non_seasonal, label='ARIMA No Seasonal')
ax[0, 1].plot(toyota_predict.index, toyota_stl, label='STL')
ax[0, 1].plot(toyota_predict.index, toyota_linear, label='Forecasting Lineal')
ax[0, 1].set_title('Predicción de precios de acciones de Toyota')
ax[0, 1].legend()

ax[1, 0].plot(tesla_test.index, tesla_test['close'], label='Datos reales')
ax[1, 0].plot(tesla_test.index, tesla_arima_seasonal[:len(tesla_test)], label='ARIMA Seasonal')
ax[1, 0].plot(tesla_test.index, tesla_arima_non_seasonal[:len(tesla_test)], label='ARIMA No Seasonal')
ax[1, 0].plot(tesla_test.index, tesla_stl[:len(tesla_test)], label='STL')
ax[1, 0].plot(tesla_test.index, tesla_linear[:len(tesla_test)], label='Forecasting Lineal')
ax[1, 0].set_title('Comparación de predicciones para Tesla (2022-2023)')
ax[1, 0].legend()

ax[1, 1].plot(toyota_test.index, toyota_test['close'], label='Datos reales')
ax[1, 1].plot(toyota_test.index, toyota_arima_seasonal[:len(toyota_test)], label='ARIMA Seasonal')
ax[1, 1].plot(toyota_test.index, toyota_arima_non_seasonal[:len(toyota_test)], label='ARIMA No Seasonal')
ax[1, 1].plot(toyota_test.index, toyota_stl[:len(toyota_test)], label='STL')
ax[1, 1].plot(toyota_test.index, toyota_linear[:len(toyota_test)], label='Forecasting Lineal')
ax[1, 1].set_title('Comparación de predicciones para Toyota (2022-2023)')
ax[1, 1].legend()

plt.tight_layout()
plt.show()